# 시맨틱 검색 - Semantic Search


## 시맨틱 검색 (Semantic Search)

오픈서치의 시맨틱 검색은 검색 쿼리의 의미를 이해하고, 그에 따라 가장 관련성 높은 결과를 사용자에게 제공하는 기능입니다. 이는 전통적인 키워드 기반 검색과 달리, 검색 쿼리의 맥락과 의미를 분석하여 보다 정확하고 관련성 높은 검색 결과를 제공합니다. 예를 들어 "아마존"이라는 단어가 상품명인지, 회사명인지, 아니면 지역명인지 등 쿼리의 의미를 파악하여 그에 맞는 검색 결과를 제공합니다.


## 신경망 검색(Neural Search)

수년 동안 고객들은 OpenSearch k-NN을 기반으로 시맨틱 검색 애플리케이션을 구축하기 위해서는 텍스트 임베딩 모델을 검색 및 수집 파이프라인에 통합하기 위해 추가적인 미들웨어를 구축해야 하는 부담이 있었습니다. 이제 Amazon SageMaker와 Amazon Bedrock와의 통합을 통해 신경망 검색을 강화하며 클러스터에서 실행되는 시맨틱 검색 파이프라인을 지원할 수 있습니다.

신경망 검색은 텍스트를 벡터로 변환하고 인덱싱 시간과 검색 시간 모두에서 벡터 검색을 용이하게 합니다. 인덱싱 중에 신경망 검색은 문서 텍스트를 벡터 임베딩으로 변환하고 텍스트와 그 벡터 임베딩을 모두 벡터 인덱스에 인덱싱합니다. 신경망 쿼리를 사용하는 경우 신경망 검색은 쿼리 텍스트를 벡터 임베딩으로 변환하고, 벡터 검색을 사용하여 쿼리와 문서 임베딩을 비교한 다음 가장 가까운 결과를 반환합니다.

문서를 인덱스에 인제스트하기 전에 문서는 기계 학습(ML) 모델을 통과하게 되며, 이 모델은 문서 필드에 대한 벡터 임베딩을 생성합니다. 검색 요청을 보내면 쿼리 텍스트나 이미지도 ML 모델을 통과하여 해당 벡터 임베딩을 생성합니다. 그런 다음 신경망 검색이 임베딩에 대한 벡터 검색을 수행하고 일치하는 문서를 반환합니다.

신경망 검색을 사용하면 OpenSearch API를 통해 인간의 언어로 검색 쿼리를 실행하고, Amazon SageMaker에서 호스팅되거나 Amazon Bedrock에서 관리하는 텍스트 임베딩을 통해 의미론적 이해와 유사성을 고려한 텍스트 임베딩을 사용하여 더 정확한 결과를 제공할 수 있습니다.


## 사전준비


필요한 패키지를 설치합니다.


In [1]:
%store -r model_id

In [2]:
print(model_id)

xGHPbpEBQH0FPvprOtAr


# 데이터 준비

키워드 검색 단계에서와 마찬가지로 데이터를 준비합니다.


In [3]:
import pandas as pd
import requests

df = pd.read_csv("./data/movies.csv", low_memory=False)
df.head(5)

,title,genre,year,date,rating,vote_count,plot,main_act,supp_act
0,변호인,드라마,2013,12.18,8.99,94574,"198 년대 초 부산. 빽 없고, 돈 없고, 가방끈도 짧은 세무 변호사 송우석(송강...",송강호|김영애|오달수|곽도원|임시완,송영창|정원중|조민기|이항나|이성민|차은재|차광수|한기중|심희섭|조완기
1,어벤져스: 엔드게임,액션|SF,2019,4.24,9.38,68923,인피니티 워 이후 절반만 살아남은 지구 마지막 희망이 된 어벤져스 먼저 떠난 그들을...,로버트 다우니 주니어|크리스 에반스|크리스 헴스워스|마크 러팔로|스칼렛 요한슨|제레...,베네딕트 컴버배치|조 샐다나|크리스 프랫|채드윅 보스만|톰 홀랜드|안소니 마키|기네...
2,명량,액션|드라마,2014,7.30,8.44,66953,"1597년 임진왜란 6년, 오랜 전쟁으로 인해 혼란이 극에 달한 조선. 무서운 속도...",최민식|류승룡|조진웅,진구|이정현|김명곤|권율|노민우|김태훈|오타니 료헤이|이승준|김강일|박보검|이해영|...
3,부산행,액션|스릴러,2016,7.20,8.00,59184,"정체불명의 바이러스가 전국으로 확산되고 대한민국 긴급재난경보령이 선포된 가운데, 열...",공유|정유미|마동석|김수안|김의성|최우식|안소희,최귀화|정석용|예수정|박명신|장혁진
4,신과함께-죄와 벌,판타지|드라마,2017,12.20,7.83,58124,"저승 법에 의하면, 모든 인간은 사후 49일 동안 7번의 재판을 거쳐야만 한다. 살...",하정우|차태현|주지훈|김향기|김동욱|마동석,오달수|임원희|디오|이준혁|예수정|장광|정해균|김수안|남일우|정지훈


데이터의 스키마를 확인합니다.


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   title       1000 non-null   object 
 1   genre       1000 non-null   object 
 2   year        1000 non-null   int64  
 3   date        1000 non-null   float64
 4   rating      1000 non-null   float64
 5   vote_count  1000 non-null   int64  
 6   plot        1000 non-null   object 
 7   main_act    1000 non-null   object 
 8   supp_act    1000 non-null   object 
dtypes: float64(2), int64(2), object(5)
memory usage: 70.4+ KB


In [5]:
from tiktoken._educational import *

# Train a BPE tokeniser on a small amount of text
enc = tiktoken.get_encoding("cl100k_base")
len(enc.encode("hello world aaaaaaaaaaaa"))

5

벡터 필드에 영화에 대한 전체적인 정보를 담기 위해 전체 컬럼을 모두 조합한 `text` 컬럼을 추가합니다. 이 `text` 필드는 이후 ingest pipeline에 의해 임베딩될 필드입니다.
이 워크샵에서 사용할 임베딩 모델인 Cohere Embed Multilingual의 Max Sequence 길이는 512입니다. 따라서 여기에 맞게 임베딩할 plot 컬럼의 최대 길이를 512로 truncate합니다.

In [6]:
import pandas as pd


def create_text(row):
    text = ""
    for col, val in row.items():
        text += f"{col}: {val} | "

    encoding = tiktoken.get_encoding("cl100k_base")

    tokens = encoding.encode(text)

    # Get the number of tokens
    num_tokens = len(tokens)
    # print(f"Number of tokens: {num_tokens}")

    # Truncate the text to 500 tokens
    max_tokens = 8000
    if num_tokens > max_tokens:
        truncated_tokens = tokens[:max_tokens]
        truncated_text = encoding.decode(truncated_tokens)
        print(f"Truncated text ({max_tokens} tokens): {truncated_text}")
        return truncated_text
    else:
        return text


# Assuming your DataFrame is called 'df'
df["text"] = df.apply(create_text, axis=1)
df.head(10)

,title,genre,year,date,rating,vote_count,plot,main_act,supp_act,text
0,변호인,드라마,2013,12.18,8.99,94574,"198 년대 초 부산. 빽 없고, 돈 없고, 가방끈도 짧은 세무 변호사 송우석(송강...",송강호|김영애|오달수|곽도원|임시완,송영창|정원중|조민기|이항나|이성민|차은재|차광수|한기중|심희섭|조완기,title: 변호인 | genre: 드라마 | year: 2013 | date: 1...
1,어벤져스: 엔드게임,액션|SF,2019,4.24,9.38,68923,인피니티 워 이후 절반만 살아남은 지구 마지막 희망이 된 어벤져스 먼저 떠난 그들을...,로버트 다우니 주니어|크리스 에반스|크리스 헴스워스|마크 러팔로|스칼렛 요한슨|제레...,베네딕트 컴버배치|조 샐다나|크리스 프랫|채드윅 보스만|톰 홀랜드|안소니 마키|기네...,title: 어벤져스: 엔드게임 | genre: 액션|SF | year: 2019 ...
2,명량,액션|드라마,2014,7.30,8.44,66953,"1597년 임진왜란 6년, 오랜 전쟁으로 인해 혼란이 극에 달한 조선. 무서운 속도...",최민식|류승룡|조진웅,진구|이정현|김명곤|권율|노민우|김태훈|오타니 료헤이|이승준|김강일|박보검|이해영|...,title: 명량 | genre: 액션|드라마 | year: 2014 | date:...
3,부산행,액션|스릴러,2016,7.20,8.00,59184,"정체불명의 바이러스가 전국으로 확산되고 대한민국 긴급재난경보령이 선포된 가운데, 열...",공유|정유미|마동석|김수안|김의성|최우식|안소희,최귀화|정석용|예수정|박명신|장혁진,title: 부산행 | genre: 액션|스릴러 | year: 2016 | date...
4,신과함께-죄와 벌,판타지|드라마,2017,12.20,7.83,58124,"저승 법에 의하면, 모든 인간은 사후 49일 동안 7번의 재판을 거쳐야만 한다. 살...",하정우|차태현|주지훈|김향기|김동욱|마동석,오달수|임원희|디오|이준혁|예수정|장광|정해균|김수안|남일우|정지훈,title: 신과함께-죄와 벌 | genre: 판타지|드라마 | year: 2017...
5,군함도,액션|드라마,2017,7.26,5.29,53895,1945년 일제강점기. 경성 반도호텔 악단장 ‘강옥’(황정민)과 그의 하나뿐인 딸 ...,황정민|소지섭|송중기|이정현|김수안,이경영|김민재|김중희|김인우|신승환|백승철|윤경호|장성범,title: 군함도 | genre: 액션|드라마 | year: 2017 | date...
6,26년,액션,2012,11.29,6.40,51885,"광주 수호파 중간보스 곽진배, 국가대표 사격선수 심미진, 서대문소속 경찰 권정혁, ...",진구|한혜진|임슬옹,배수빈|이경영|장광|이미도|조덕제|김의성|안석환|민복기|구성환|김민재|최귀화|김태수|장영,title: 26년 | genre: 액션 | year: 2012 | date: 11...
7,인터스텔라,SF,2014,11.06,9.10,50721,세계 각국의 정부와 경제가 완전히 붕괴된 미래가 다가온다. 지난 2 세기에 범한 잘...,매튜 맥커너히|앤 해서웨이|마이클 케인|제시카 차스테인,케이시 애플렉|웨스 벤틀리|토퍼 그레이스|매켄지 포이|엘렌 버스틴|존 리스고|빌 어...,title: 인터스텔라 | genre: SF | year: 2014 | date: ...
8,택시운전사,드라마|가족,2017,8.02,9.03,49661,택시운전사 만섭(송강호)은 외국손님을 태우고 광주에 갔다 통금 전에 돌아오면 밀린 ...,송강호|토마스 크레취만|유해진|류준열,박혁권|최귀화|차순배|신담수|류성현|엄태구|박민희|이정은|유은미|권순준|윤석호|허정...,title: 택시운전사 | genre: 드라마|가족 | year: 2017 | da...
9,극한직업,코미디,2019,1.23,8.50,47522,"불철주야 달리고 구르지만 실적은 바닥, 급기야 해체 위기를 맞는 마약반! 더 이상 ...",류승룡|이하늬|진선규|이동휘|공명,신하균|오정세|김의성|송영규|양현민|허준석|장진희|김종수|이중옥,title: 극한직업 | genre: 코미디 | year: 2019 | date: ...


### OpenSearch 도메인에 연결


In [7]:
from opensearchpy import OpenSearch, RequestsHttpConnection
import json

# OpenSearch 연결 설정
host = 'localhost'
port = 9200
auth = ('admin', 'TestUser2@')  # 초기 설정한 어드민 비밀번호 사용

aos_client = OpenSearch(
    hosts=[{'host': host, 'port': port}],
    http_auth=auth,
    use_ssl=True,
    verify_certs=False,
    ssl_show_warn=False,
)

## 데이터 임베딩을 위한 Ingest Pipeline 생성


모델 배포 과정을 참고하여 배포된 모델의 ID를 확인하고 아래와 같이 변수에 초기화합니다.


In [8]:
import requests

# search_model = {"query": {"match": {"name": "OpenSearch-Cohere"}}, "size": 10}

# response = requests.get(
#     "https://" + aos_host + "/_plugins/_ml/models/_search", auth=auth, json=search_model
# )
# model_info = json.loads(response.text)
# model_id = model_info["hits"]["hits"][0]["_id"]
# model_id

## 인제스트 파이프라인 생성

영화의 줄거리 정보를 가진 특정 필드("plot")에 대한 벡터 임베딩을 생성하는 파이프라인을 설정합니다. 이러한 임베딩은 검색 인덱스 필드("vector_field")에 저장되며, 효율적인 유사성 검색 및 검색 작업에 사용될 수 있습니다.


In [9]:
base_url = "https://localhost:9200/_plugins/_ml"
host = 'localhost'
port = 9200
auth = ('admin', 'TestUser2@')  # 초기 설정한 어드민 비밀번호 사용

In [10]:
response = requests.get(
    base_url + "/models/" + model_id,
    auth=auth,
    verify=False
)

In [11]:
response.json()

{'name': 'OpenAI text embedding model',
 'model_group_id': 'sGHLSpEBQH0FPvpr6syc',
 'algorithm': 'REMOTE',
 'model_version': '3',
 'description': 'This is OpenAI Embeddings text-embedding-3-small',
 'model_state': 'DEPLOYED',
 'created_time': 1724140960299,
 'last_updated_time': 1724140972669,
 'last_deployed_time': 1724140972669,
 'auto_redeploy_retry_times': 0,
 'planning_worker_node_count': 1,
 'current_worker_node_count': 1,
 'planning_worker_nodes': ['tz0Gz8FBRMul2wMCgblyQQ'],
 'deploy_to_all_nodes': True,
 'is_hidden': False,
 'connector_id': 'wmHLbpEBQH0FPvpr89Du'}

In [12]:
pipeline = {
    "description": "OpenAI text embedding",
    "processors": [
        {
            "text_embedding": {
                "model_id": model_id,
                "field_map": {
                    "textInput": "vector_field",
                },
            }
        }
    ],
}
# pipeline = {
#     "description": "Test copy pipeline",
#     "processors": [
#         {
#             "set": {
#                 "field": "vector_field",
#                 "value": "{{textInput}}"  # textInput 필드의 내용을 vector_field로 복사
#             }
#         }
#     ],
# }

pipeline_id = "movie_plot_embedding_pipeline"
aos_client.ingest.delete_pipeline(id=pipeline_id)
aos_client.ingest.put_pipeline(id=pipeline_id, body=pipeline)

{'acknowledged': True}

In [13]:
# 문서 인덱싱
document = {
    "inputText": "Example movie plot goes here"
}

response = aos_client.index(
    index="movie_plots_index",
    id="1",
    body=document,
    pipeline=pipeline_id
)

print(response)

{'_index': 'movie_plots_index', '_id': '1', '_version': 5, 'result': 'updated', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 4, '_primary_term': 1}


In [14]:
response = aos_client.get(index="movie_plots_index", id="1")
vector_field = response['_source'].get('vector_field')

print(response)

{'_index': 'movie_plots_index', '_id': '1', '_version': 5, '_seq_no': 4, '_primary_term': 1, 'found': True, '_source': {'inputText': 'Example movie plot goes here'}}


## 인덱스 생성

movie_semantic 인덱스를 생성합니다. 아래 세팅 및 맵핑 정보 중 중요한 것은 다음과 같습니다.

-   index.knn: KNN 검색을 위해 True로 설정합니다.
-   default_pipeline: 위 단계에서 생성한 pipeline_id를 제공합니다.
-   index.knn.space_type: 임베딩 벡터끼리의 유사도를 파악할 때 사용할 알고리즘을 cosinesimil로 지정합니다


In [16]:
index_name = "movie_semantic"

# aos_client.indices.delete(index=index_name)

movie_semantic = {
    "settings": {
        "max_result_window": 15000,
        "analysis": {"analyzer": {"analysis-nori": {"type": "nori", "stopwords": "_korean_"}}},
        "index.knn": True,
        "default_pipeline": pipeline_id,
        "index.knn.space_type": "l2",
    },
    "mappings": {
        "properties": {
            "date": {
                "type": "float",
            },
            "genre": {
                "type": "text",
            },
            "main_act": {
                "type": "text",
                "fields": {"keyword": {"type": "keyword", "ignore_above": 256}},
            },
            "plot": {
                "type": "text",
            },
            "rating": {
                "type": "float"
            },
            "supp_act": {
                "type": "text",
                "fields": {"keyword": {"type": "keyword", "ignore_above": 256}},
            },
            "title": {
                "type": "text",
                "fields": {"keyword": {"type": "keyword", "ignore_above": 256}},
            },
            "vote_count": {
                "type": "long"
            },
            "text": {
                "type": "text",
            },
            "year": {
                "type": "long"
            },
            "vector_field": {
                "type": "knn_vector",
                "dimension": 1536,
                "method": {"name": "hnsw", "space_type": "l2", "engine": "faiss"},
                "store": True,
            },
        }
    },
}

aos_client.indices.delete(index=index_name)
aos_client.indices.create(index=index_name, body=movie_semantic)

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'movie_semantic'}

In [17]:
aos_client.indices.get(index=index_name)

{'movie_semantic': {'aliases': {},
  'mappings': {'properties': {'date': {'type': 'float'},
    'genre': {'type': 'text'},
    'main_act': {'type': 'text',
     'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}},
    'plot': {'type': 'text'},
    'rating': {'type': 'float'},
    'supp_act': {'type': 'text',
     'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}},
    'text': {'type': 'text'},
    'title': {'type': 'text',
     'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}},
    'vector_field': {'type': 'knn_vector',
     'store': True,
     'dimension': 1536,
     'method': {'engine': 'faiss',
      'space_type': 'l2',
      'name': 'hnsw',
      'parameters': {}}},
    'vote_count': {'type': 'long'},
    'year': {'type': 'long'}}},
  'settings': {'index': {'replication': {'type': 'DOCUMENT'},
    'number_of_shards': '1',
    'provided_name': 'movie_semantic',
    'knn.space_type': 'l2',
    'max_result_window': '15000',
    'default_pipeli

## 데이터 인제스트

키워드 검색과 동일하게 데이터를 인제스트합니다. 위 단계에서 인덱스를 생성할 때 INGEST PIPELINE을 설정했기 때문에 직접 임베딩 모델을 호출하여 데이터를 벡터로 변환하지 않아도 됩니다. 단, 데이터가 인제스트될 때 임베딩 모델을 호출해야 하는 단계가 있기 때문에 parallel_bulk의 병렬도가 높으면 에러가 발생할 있습니다. 여기서는 `thread_count`와 `queue_size`를 각각 1로 낮춰줍니다. 이 단계가 완료되는데는 약 4~5분 정도 소요됩니다.


In [18]:
from tqdm import tqdm
from opensearchpy import helpers

json_data = df.to_json(orient="records", lines=True)
docs = json_data.split("\n")[:-1]  # To remove the last empty line

In [19]:
df.head(3)

,title,genre,year,date,rating,vote_count,plot,main_act,supp_act,text
0,변호인,드라마,2013,12.18,8.99,94574,"198 년대 초 부산. 빽 없고, 돈 없고, 가방끈도 짧은 세무 변호사 송우석(송강...",송강호|김영애|오달수|곽도원|임시완,송영창|정원중|조민기|이항나|이성민|차은재|차광수|한기중|심희섭|조완기,title: 변호인 | genre: 드라마 | year: 2013 | date: 1...
1,어벤져스: 엔드게임,액션|SF,2019,4.24,9.38,68923,인피니티 워 이후 절반만 살아남은 지구 마지막 희망이 된 어벤져스 먼저 떠난 그들을...,로버트 다우니 주니어|크리스 에반스|크리스 헴스워스|마크 러팔로|스칼렛 요한슨|제레...,베네딕트 컴버배치|조 샐다나|크리스 프랫|채드윅 보스만|톰 홀랜드|안소니 마키|기네...,title: 어벤져스: 엔드게임 | genre: 액션|SF | year: 2019 ...
2,명량,액션|드라마,2014,7.30,8.44,66953,"1597년 임진왜란 6년, 오랜 전쟁으로 인해 혼란이 극에 달한 조선. 무서운 속도...",최민식|류승룡|조진웅,진구|이정현|김명곤|권율|노민우|김태훈|오타니 료헤이|이승준|김강일|박보검|이해영|...,title: 명량 | genre: 액션|드라마 | year: 2014 | date:...


In [20]:
def _generate_data():
    for doc in docs:
        yield {"_index": index_name, "_source": doc}

succeeded = []
failed = []
for success, item in helpers.parallel_bulk(
    aos_client, actions=_generate_data(), chunk_size=10, thread_count=1, queue_size=1
):
    if success:
        succeeded.append(item)
    else:
        failed.append(item)

데이터 인제스트가 잘 마무리되었는지 확인합니다.


In [21]:
# Refresh the index to make the changes visible
aos_client.indices.refresh(index=index_name)

count = aos_client.count(index=index_name)
print(count)

{'count': 1000, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}}


# 키워드 검색과 시맨틱 검색 결과 비교


## 키워드 검색을 위한 함수 생성

키워드 단계에서 정의한 키워드 검색 함수와 동일한 함수를 생성합니다.


In [22]:
def keyword_search(query_text):
    query = {
        "size": 10,
        "_source": {"excludes": ["text", "vector_field"]},
        "query": {
            "multi_match": {
                "query": query_text,
                "fields": ["title", "plot", "genre", "main_act"],
            }
        },
    }

    res = aos_client.search(index=index_name, body=query)

    query_result = []
    for hit in res["hits"]["hits"]:
        row = [
            hit["_score"],
            hit["_source"]["title"],
            hit["_source"]["plot"],
            hit["_source"]["genre"],
            hit["_source"]["rating"],
            hit["_source"]["main_act"],
        ]
        query_result.append(row)

    query_result_df = pd.DataFrame(
        data=query_result, columns=["_score", "title", "plot", "genre", "rating", "main_act"]
    )
    display(query_result_df)

## 시맨팀 검색을 위한 함수 생성

뉴럴 검색을 위한 함수를 생성합니다. 주목해야할 부분은 다음과 같습니다.

1. **`"_source": {"excludes": ["vector_field"]}`**: 검색 결과에서 **`vector_field`** 필드를 제외한 모든 필드를 반환합니다. 이는 벡터 데이터가 크기 때문에 전송 비용을 줄이기 위함입니다.
2. **`"query": { ... }`**: 실제 검색 쿼리를 정의합니다.
3. **`"neural": { ... }`**: 벡터 검색을 수행하기 위한 쿼리 유형입니다.
4. **`"vector_field": "vector_field_name"`**: 벡터 데이터가 저장된 필드 이름입니다.
5. **`"query_text": query_text`**: 검색할 텍스트 쿼리입니다.
6. **`"model_id": model_id`**: 벡터 임베딩을 생성하는 데 사용된 모델의 ID입니다.


In [44]:
def semantic_search(query_text):
    query = {
        "size": 10,
        "_source": {"excludes": ["text", "vector_field"]},
        "query": {
            "neural": {
                "vector_field": {
                    "query_text": query_text,
                    # "inputText": query_text,
                    "model_id": model_id,
                    "k": 10
                }
            },
        },
        # "parameters": {"inputText": query_text}
    }

    res = aos_client.search(index=index_name, body=query)

    query_result = []
    for hit in res["hits"]["hits"]:
        row = [
            hit["_score"],
            hit["_source"]["title"],
            hit["_source"]["plot"],
            hit["_source"]["genre"],
            hit["_source"]["rating"],
            hit["_source"]["main_act"],
        ]
        query_result.append(row)

    query_result_df = pd.DataFrame(
        data=query_result, columns=["_score", "title", "plot", "genre", "rating", "main_act"]
    )
    display(query_result_df)

## 결과 비교

자연어 기반의 쿼리를 작성하고 키워드 검색과 시맨틱 검색의 결과를 비교해봅니다


In [45]:
query_text = "우주에서 벌어지는 전쟁 이야기를 다룬 영화 소개해줘"

In [46]:
keyword_search(query_text)

,_score,title,plot,genre,rating,main_act
0,7.102402,황금나침반,‘살아 있는 모든 존재를 멸망으로 이끌 전쟁이 일어날 것’이라는 예언의 중심에 있는...,판타지|모험|액션|드라마|가족|스릴러,6.25,다니엘 크레이그|니콜 키드먼|에바 그린|다코타 블루 리차드
1,7.030644,영화는 영화다,영화를 촬영하던 배우 장수타(강지환 扮)는 액션씬에서 욱하는 성질을 참지 못해 상대...,액션|범죄|드라마|느와르,8.85,소지섭|강지환
2,6.071900,명탐정 코난: 천공의 난파선,명쾌한 추리와 시원한 액션으로 사건을 해결했던 코난. 이번에는 세계 최대 비행선에서...,애니메이션|모험|가족,9.23,타카야마 미나미|야마자키 와카나|야마구치 캇페이|코야마 리키야|하야시바라 메구미|호...
3,6.002809,더 넌,루마니아의 젊은 수녀가 자살하는 사건을 의뢰 받아 바티칸에서 파견된 버크 신부와 아...,공포|미스터리|스릴러,6.66,타이사 파미가|데미안 비쉬어|보니 아론스
4,5.699466,우주 전쟁,레이 페리어(톰 크루즈 분)는 이혼한 항만 근로자로 아무런 희망 없이 매일을 살아간...,SF|드라마|스릴러,7.31,톰 크루즈
5,5.646401,경성학교: 사라진 소녀들,"외부와 단절된 경성의 한 기숙학교. 어느 날부터, 학생들이 하나 둘 이상 증세를 보...",미스터리|드라마,6.12,박보영|엄지원|박소담
6,5.629383,오빠생각,"전쟁으로 소중한 가족도, 지켜야 할 동료도 모두 잃은 군인 ‘한상렬’(임시완). 우...",드라마|전쟁,8.51,임시완|고아성
7,5.601114,램페이지,"유인원 전문가 ‘데이비스(드웨인 존슨)’는 사람들은 멀리하지만, 어릴 때 구조해서 ...",액션|모험,8.28,드웨인 존슨|제프리 딘 모건|나오미 해리스|말린 애커맨
8,5.429001,곡성(哭聲),낯선 외지인(쿠니무라 준)이 나타난 후 벌어지는 의문의 연쇄 사건들로 마을이 발칵 ...,미스터리|스릴러|드라마,7.61,곽도원|황정민|쿠니무라 준|천우희|김환희
9,5.404394,엑스맨: 다크 피닉스,어린 시절 비극적 교통사고로 자신의 능력을 알게 된 진 그레이는 자비에 영재학교에서...,액션|모험|SF,5.96,제임스 맥어보이|마이클 패스벤더|제니퍼 로렌스|니콜라스 홀트|소피 터너|타이 쉐리던...


In [48]:
query = {
    "size": 10,
    "_source": {"excludes": ["text", "vector_field"]},
    "query": {
        "neural": {
            "vector_field": {
                "query_text": query_text,
                # "inputText": query_text,
                "model_id": model_id,
                "k": 10
            }
        },
    },
    # "parameters": {"inputText": query_text}
}

res = aos_client.search(index=index_name, body=query)

RequestError: RequestError(400, 'illegal_argument_exception', 'Some parameter placeholder not filled in payload: inputText')

In [47]:
semantic_search(query_text)
# print(index_name)

RequestError: RequestError(400, 'parsing_exception', 'Unknown key for a VALUE_STRING in [inputText].')

In [27]:
%store model_id
%store index_name

Stored 'model_id' (str)
Stored 'index_name' (str)
